# 基于 PyTorch 的联邦学习自定义 loss function 教程
## 引言
### 背景
在联邦学习中，尤其是监督学习中，我们常常需要使用损失函数监督模型的训练；通过之前的[入门教程](https://www.secretflow.org.cn/docs/secretflow/latest/zh-Hans/tutorial/Federated_Learning_with_Pytorch_backend), 我们已经展示如何通过 `secretflow_fl.ml.nn.core.torch.TorchModel` 调用 `torch.nn.CrossEntropyLoss` ，依此类推，我们可以调用 [torch.nn loss function](https://pytorch.org/docs/stable/nn.html#loss-functions) 中的任意损失函数。然而，当我们需要根据自己的任务自定义损失函数时，需要怎样做呢？本教程将回答这一问题。
### 教程提醒
注意，本自定义 loss function 教程主要关注输入形式为$(\hat{y},y)$的损失函数，而不讨论超出此范围的自定义损失函数。
具体到本教程，本教程将给出如何自定义实现

$$
Loss(\hat{y},y) = 0.8*CEL(\hat{y},y) + 0.2*MSE(\hat{y},y)
$$

其中，$CEL$ 表示 [cross entropy loss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) ，$MSE$ 表示[mean squared error](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss)，对于其他的损失函数组合形式，您可以自行定义和组合。

再度提醒，本教程只是作为教程示例，展示代码的实现，而不作为实际生产应用的模型训练指导。

让我们开始吧！

## 基础教程
为突出重点，简化教程，本教程将以 [使用Pytorch后端来进行联邦学习](https://www.secretflow.org.cn/docs/secretflow/latest/zh-Hans/tutorial/Federated_Learning_with_Pytorch_backend) 为基础，重点突出自定义损失函数的做法。所以，为了让代码能够顺利运行，让我们先把之前的代码复制过来。因此如果您对原教程非常熟悉，则不需要再阅读这部分代码。

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import secretflow as sf

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))

# In case you have a running secretflow runtime already.
sf.shutdown()

sf.init(['alice', 'bob', 'charlie'], address='local')
alice, bob, charlie = sf.PYU('alice'), sf.PYU('bob'), sf.PYU('charlie')

The version of SecretFlow: 1.4.0.dev20231225


2024-01-10 09:34:35,376	INFO worker.py:1538 -- Started a local Ray instance.


In [3]:
from secretflow_fl.ml.nn.core.torch import (
    metric_wrapper,
    optim_wrapper,
    BaseModule,
    TorchModel,
)
from secretflow_fl.ml.nn import FLModel
from torchmetrics import Accuracy, Precision
from secretflow.security.aggregation import SecureAggregator
from secretflow_fl.utils.simulation.datasets_fl import load_mnist
from torch import nn, optim
from torch.nn import functional as F

2024-01-10 09:34:37.234123: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /content/conda-env/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-01-10 09:34:38.012003: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /content/conda-env/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024-01-10 09:34:38.012080: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /content/conda-env/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2024

In [4]:
class ConvNet(BaseModule):
    """Small ConvNet for MNIST."""

    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3)
        self.fc_in_dim = 192
        self.fc = nn.Linear(self.fc_in_dim, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 3))
        x = x.view(-1, self.fc_in_dim)
        x = self.fc(x)
        return F.softmax(x, dim=1)

## 自定义损失函数
如前所述，我们将自定义损失函数：

$$
Loss(\hat{y},y) = 0.8*CEL(\hat{y},y) + 0.2*MSE(\hat{y},y)
$$

其中，$CEL$ 表示 [cross entropy loss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)，$MSE$ 表示 [mean squared error](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss)

为实现这一个自定义损失函数，我们存在两种实现方式，一种是继承[torch.nn.module](https://github.com/pytorch/pytorch/tree/main/torch/nn/modules) 的类，另外一种直接定义函数。
### 继承 torch.nn.module
#### 继承介绍
我们需要自行编写一个继承自 [torch.nn.module](https://github.com/pytorch/pytorch/tree/main/torch/nn/modules) 的类，而且至少实现两个基础的函数：`__init__` 和 `forward`，其中:
- `__init__` 执行该类的初始化部分代码，本教程我们对基础损失函数 `CrossEntropyLoss` 和 `MSELoss` 进行了初始化的操作
- `forward`  执行该类的调用时的运算代码，也就是自定义损失函数的运算逻辑，此处我们对上面所提及的自定义函数进行了实现

#### 实现自定义类

In [5]:
class CustomLossFunction(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.cross_entropy_loss = nn.CrossEntropyLoss(reduction='sum')
        self.mse_loss = nn.MSELoss()

    def forward(self, input, target):
        return 0.8 * self.cross_entropy_loss(input, target) + 0.2 * self.mse_loss(
            input, target
        )

### 直接自定义损失函数
#### 自定义函数介绍
我们也可以直接定义损失函数，对于同一实现，直接实现如下：
#### 自定义函数实现

In [6]:
def my_loss_function(input, target):
    cross_entropy_loss = nn.CrossEntropyLoss(reduction='sum')
    mse_loss = nn.MSELoss()
    return 0.8 * cross_entropy_loss(input, target) + 0.2 * mse_loss(input, target)

## 指定自定义损失函数
### 继承 torch.nn.module
当我们通过继承 torch.nn.module 实现自定义函数时，我们可以在下面的单元格里，通过
``
loss_fn = CustomLossFunction
``
指定我们自定义的损失函数。

In [7]:
# here we use the loss function we defined above
loss_fn = CustomLossFunction

optim_fn = optim_wrapper(optim.Adam, lr=1e-2)
model_def = TorchModel(
    model_fn=ConvNet,
    loss_fn=loss_fn,
    optim_fn=optim_fn,
    metrics=[
        metric_wrapper(Accuracy, task="multiclass", num_classes=10, average='micro'),
        metric_wrapper(Precision, task="multiclass", num_classes=10, average='micro'),
    ],
)

### 直接自定义损失函数
当我们通过直接自定义损失函数实现时，我们可以在下面的单元格里，通过
``
loss_fn = my_loss_function
``
指定我们自定义的损失函数。

In [8]:
# here we use the loss function we defined above
loss_fn = my_loss_function

optim_fn = optim_wrapper(optim.Adam, lr=1e-2)
model_def = TorchModel(
    model_fn=ConvNet,
    loss_fn=loss_fn,
    optim_fn=optim_fn,
    metrics=[
        metric_wrapper(Accuracy, task="multiclass", num_classes=10, average='micro'),
        metric_wrapper(Precision, task="multiclass", num_classes=10, average='micro'),
    ],
)

## 小结
通过本教程，我们将学会如何基于 PyTorch 在SecretFlow 中自定义实现输入形式为 $(\hat{y},y)$ 的损失函数。